# In this file we simply are interested to find out which k-space column lines are acquired in which echo trains. 
And how many echo trains are there?


In [4]:
import numpy as np
import twixtools
from pathlib import Path
# That would be this function, but it is not used in the code above.
import ismrmrd
from assets.operations_kspace import get_first_acquisition, echo_train_count, echo_train_length, get_num_averages
from assets.util import setup_logger


# All patient IDs to consider for Uncertainty Quantification
pat_ids = [
    # '0003_ANON5046358',
    # '0004_ANON9616598',
    # '0005_ANON8290811',
    # '0006_ANON2379607',
    # '0007_ANON1586301',
    # '0008_ANON8890538',
    # '0010_ANON7748752',
    # '0011_ANON1102778',
    # '0012_ANON4982869',
    # '0013_ANON7362087',
    # '0014_ANON3951049',
    # '0015_ANON9844606',
    # '0018_ANON9843837',
    # '0019_ANON7657657',
    # '0020_ANON1562419',
    # '0021_ANON4277586',
    # '0023_ANON6964611',
    # '0024_ANON7992094',
    # '0026_ANON3620419',
    # '0027_ANON9724912',
    # '0028_ANON3394777',
    # '0029_ANON7189994',
    # '0030_ANON3397001',
    # '0031_ANON9141039',
    # '0032_ANON7649583',
    # '0033_ANON9728185',
    # '0035_ANON3474225',
    # '0036_ANON0282755',
    # '0037_ANON0369080',
    # '0039_ANON0604912',
    # '0042_ANON9423619',
    # '0043_ANON7041133',
    # '0044_ANON8232550',
    # '0045_ANON2563804',
    # '0047_ANON3613611',
    # '0048_ANON6365688',
    # '0049_ANON9783006',
    # '0051_ANON1327674',
    # '0052_ANON9710044',
    # '0053_ANON5517301',
    # '0055_ANON3357872',
    # '0056_ANON2124757',
    # '0057_ANON1070291',
    # '0058_ANON9719981',
    # '0059_ANON7955208',
    # '0061_ANON7642254',
    # '0062_ANON0319974',
    # '0063_ANON9972960',
    # '0064_ANON0282398',
    # '0067_ANON0913099',
    # '0068_ANON7978458',
    # '0069_ANON9840567',
    # '0070_ANON5223499',
    # '0071_ANON9806291',
    # '0073_ANON5954143',
    # '0075_ANON5895496',
    # '0076_ANON3983890',
    # '0077_ANON8634437',
    # '0078_ANON6883869',
    # '0079_ANON8828023',
    # '0080_ANON4499321',
    # '0081_ANON9763928',
    # '0082_ANON6073234',
    # '0083_ANON9898497',
    # '0084_ANON6141178',
    # '0085_ANON4535412',
    # '0086_ANON8511628',
    # '0087_ANON9534873',
    # '0088_ANON9892116',
    # '0089_ANON9786899',
    # '0090_ANON0891692',
    # '0092_ANON9941969',
    # '0093_ANON9728761',
    # '0094_ANON8024204',
    # '0095_ANON4189062',
    # '0097_ANON5642073',
    # '0103_ANON8583296',
    # '0104_ANON7748630',
    # '0105_ANON9883201',
    # '0107_ANON4035085',
    # '0108_ANON0424679',
    # '0109_ANON9816976',
    # '0110_ANON8266491',
    # '0111_ANON9310466',
    # '0112_ANON3210850',
    # '0113_ANON9665113',
    # '0115_ANON0400743',
    # '0116_ANON9223478',
    # '0118_ANON7141024',
    # '0119_ANON3865800',
    # '0120_ANON7275574',
    # '0121_ANON9629161',
    # '0123_ANON7265874',
    # '0124_ANON8610762',
    # '0125_ANON0272089',
    # '0126_ANON4747182',
    # '0127_ANON8023509',
    # '0128_ANON8627051',
    # '0129_ANON5344332',
    # '0135_ANON9879440',
    # '0136_ANON8096961',
    # '0137_ANON8035619',
    # '0138_ANON1747790',
    # '0139_ANON2666319',
    # '0140_ANON0899488',
    # '0141_ANON8018038',
    # '0142_ANON7090827',
    # '0143_ANON9752849',
    # '0144_ANON2255419',
    # '0145_ANON0335209',
    # '0146_ANON7414571',
    # '0148_ANON9604223',
    # '0149_ANON4712664',
    # '0150_ANON5824292',
    # '0152_ANON2411221',
    # '0153_ANON5958718',
    # '0155_ANON7828652',
    # '0157_ANON9873056',
    # '0159_ANON9720717',
    '0160_ANON3504149',
]

In [ ]:
def extract_echo_trains(twix_file_path, etl=25):
    """
    Extract echo train information from a Siemens raw file using twixtools.
    
    Parameters:
        twix_file_path (str or Path): Path to the raw Siemens file (.mrd or similar).
        etl (int): Echo Train Length; default is 25.
        
    Returns:
        dict: A mapping where each key is the acquisition index (line number) and the value is its echo train number.
    """
    twix_obj = twixtools.read_twix(twix_file_path)
    assert hasattr(twix_obj, 'image_header'), "Twix object does not contain image_header attribute"
    
    echo_trains = {}
    for idx, header in enumerate(twix_obj.image_header):
        # Option 1: Try to get explicit echo train info if stored in header (field name may vary)
        echo_train = header.get('echoTrain', None)
        # Option 2: If not available, assume sequential ordering with constant ETL
        if echo_train is None:
            echo_train = idx // etl
        echo_trains[idx] = echo_train
    return echo_trains


def print_echo_train_mapping(echo_trains):
    """
    Print a mapping of acquisition indices to their echo train numbers.
    
    Parameters:
        echo_trains (dict): Dictionary mapping acquisition index to echo train number.
    """
    print("Acquisition Index -> Echo Train")
    for idx, et in echo_trains.items():
        print(f"{idx} -> {et}")

# Example usage:
# twix_file = "path/to/your/file.mrd"
# echo_train_map = extract_echo_trains(twix_file, etl=25)
# print_echo_train_mapping(echo_train_map)


Explore Twixtools for a moment to understand basic functionality

In [ ]:
roots = {
    'raw_kspace': Path(f'E:/01_data/01_prostate_raw_kspace_umcg_p1'),
    '0160_mrd_fpath': Path(f'F:/01_data/02_processed_pst_ksp_umcg_p2/data/pat_data/0160_ANON3504149/mrds/meas_MID00150_FID60477_t2_tse_tra_p2_384-out_2.mrd')
}

pat_id = '0003'
kspace_dir = roots['raw_kspace'] / f'{pat_id}_patient_umcg_done/kspaces'

print(f"Looking for kspace files in {kspace_dir}")

# find all files that look like: meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat. The ID is not important, but the rest is.
print(f"Files found: {len(list(kspace_dir.glob('meas_MID*_FID*_t2_tse_traobl_p2_384.dat')))}")

t2_tse_files = list(kspace_dir.glob('meas_MID*_FID*_t2_tse_traobl_p2_384.dat'))
for dat_file in t2_tse_files:
    print(f"Processing {dat_file}")
    file_size = dat_file.stat().st_size  # size in bytes
    print(f"File size: {file_size / (1024**2):.2f} MB")
    if file_size < 1 * 1024**3:
        print(f"Skipping {dat_file}, size {file_size} bytes is smaller than 1GB.")
        continue
    print(f"Processing {dat_file} (size: {file_size} bytes)")


Looking for kspace files in E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces
Files found: 2
Processing E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00400_FID373322_t2_tse_traobl_p2_384.dat
File size: 0.87 MB
Skipping E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00400_FID373322_t2_tse_traobl_p2_384.dat, size 915456 bytes is smaller than 1GB.
Processing E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat
File size: 3579.34 MB
Processing E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat (size: 3753208832 bytes)


In [ ]:

def find_large_kspace_files(pat_id: str, kspace_dat_path: Path, pattern='meas_MID*_FID*_t2_tse_traobl_p2_384.dat', min_size=1 * 1024**3):
    """
    Find and return kspace files for a given patient that match the glob pattern and are larger than min_size.

    Parameters:
        pat_id (str): Patient identifier.
        kspace_dat_path (Path): Path to the directory containing kspace files.
        pattern (str): Glob pattern to search for.
        min_size (int): Minimum file size in bytes (default is 1GB).

    Returns:
        list: List of file paths (Path objects) that satisfy the criteria.
    """
    kspace_dir = kspace_dat_path / f'{pat_id}_patient_umcg_done/kspaces'
    valid_files = []
    for dat_file in kspace_dir.glob(pattern):
        file_size = dat_file.stat().st_size
        if file_size >= min_size:
            valid_files.append(dat_file)
            print(f"Found valid file: {dat_file}, size: {file_size / (1024**2):.2f} MB")
        else:
            print(f"Skipping {dat_file}, size {file_size} bytes is smaller than 1GB.")
    
    print(f"Found {len(valid_files)} valid kspace files for patient {pat_id} in {kspace_dir}")
    return valid_files


def import_kspace(mdb_list):
    """
    Read image data from a list of mdbs and sort into a 5D k-space array.
    
    The resulting array has shape:
        [n_avg, n_part, n_line, n_channel, n_column]
    where:
      - n_avg: number of averages (cAve),
      - n_part: number of partitions (cPar),
      - n_line: number of phase-encode lines (cLin),
      - n_channel, n_column: dimensions from the data shape.
      
    This function sums the data from each acquisition into the proper bin,
    which takes care of averaging if multiple acquisitions per index exist.
    
    Parameters:
        mdb_list (list): List of mdb objects.
        
    Returns:
        np.ndarray: 5D numpy array containing the sorted k-space data.
    """
    image_mdbs = [mdb for mdb in mdb_list if mdb.is_image_scan()]
    
    n_line = 1 + max(mdb.cLin for mdb in image_mdbs)
    n_part = 1 + max(mdb.cPar for mdb in image_mdbs)
    n_ave  = 1 + max(mdb.cAve for mdb in image_mdbs)  # average dimension
    n_channel, n_column = image_mdbs[0].data.shape

    print(f'Found nline to be: {n_line}')
    print(f'Found npart to be: {n_part}')
    print(f'Found nAve to be: {n_ave}')
    print(f'Found nchannel to be: {n_channel}')
    print(f'Found ncolumn to be: {n_column}')

    # Initialize a 5D array: average, partition, line, channel, column
    out = np.zeros([n_ave, n_part, n_line, n_channel, n_column], dtype=np.complex64)
    
    for mdb in image_mdbs:
        # The '+=' operator handles cases where multiple acquisitions fall into the same bin.
        out[mdb.cAve, mdb.cPar, mdb.cLin] += mdb.data

    return out


def extract_echo_trains_idxs(mdb_list, etl: int) -> dict:
    image_mdbs = [mdb for mdb in mdb_list if mdb.is_image_scan()]
    if not image_mdbs:
        return {}
    
    # Total lines as given by cLin counters (0-indexed)
    n_line_total = 1 + max(mdb.cLin for mdb in image_mdbs)
    print(f'Found nline_total to be: {n_line_total}')

    num_echo_trains = n_line_total // etl  # assuming it divides evenly
    print(f'Number of echo trains: {num_echo_trains}')
    
    mapping = {}
    # Map each acquired line (starting at line=1 up to n_line_total-1)
    for line in range(1, n_line_total):
        index = line - 1  # convert to 0-index for mapping
        echo_train = index // etl
        line_within_echo = index % etl
        mapping[line] = (echo_train, line_within_echo)
    
    return {
        'total_lines': n_line_total,
        'num_echo_trains': num_echo_trains,
        'mapping': mapping
    }
    

def calculate_first_average_echo_mapping(mdb_list, etl: int) -> dict:
    """
    Calculate echo train mapping for the first average from a list of mdb objects.
    
    This function filters the mdb objects for image scans in the first average (cAve == 0),
    and then computes a mapping from each echo train index to a list of acquired line indices
    (cLin) that belong to that echo train. The echo train index is calculated as:
    
        echo_train = cLin // etl
    
    Parameters:
        mdb_list (list): List of mdb objects.
        etl (int): Echo Train Length.
    
    Returns:
        dict: A dictionary mapping each echo train index (int) to a sorted list of acquired
              line indices (list of ints) from the first average.
    """
    # Filter for image scans in the first average
    first_avg = [mdb for mdb in mdb_list if mdb.is_image_scan() and mdb.cAve == 0]
    
    mapping = {}
    for mdb in first_avg:
        echo_train = mdb.cLin // etl
        mapping.setdefault(echo_train, []).append(mdb.cLin)
    
    # Sort the acquired line indices for each echo train
    for key in mapping:
        mapping[key].sort()
        print(f"Echo train {key}: {mapping[key]}")

    return mapping


def process_patient(pat_id: str, kspace_dat_path: Path):
    """
    Process a single patient by finding valid kspace files and summarizing each using twixtools.

    Parameters:
        pat_id (str): Patient identifier.
        kspace_dat_path (Path): Path to the directory containing kspace files.
    """
    print(f"\nProcessing patient: {pat_id}")
    valid_files = find_large_kspace_files(pat_id, kspace_dat_path)
    if not valid_files:
        print(f"No valid kspace files found for patient {pat_id}")
        return
    for file_path in valid_files:
        print(f"Processing {file_path}")

        twix_obj = twixtools.read_twix(str(file_path), parse_geometry=False, parse_data=True)
        print(f"Number of elements in twix_obj = {len(twix_obj)}")
        for k, v in twix_obj[-1].items():
            print(f"  {k}: value to big for printing")

        imaging_data = twix_obj[-1]
        mapping = calculate_first_average_echo_mapping(imaging_data['mdb'], etl=25)
        idxs_echo_trains = extract_echo_trains_idxs(imaging_data['mdb'], etl=25)

        print("Start of SETS")
        for k, v in mapping.items():
            # print(f"  {k}: {v}")
            print(f"Set of {k}: {set(v)}, len: {len(set(v))}")
        # kspace_array = import_kspace(imaging_data['mdb'])
        # print("K-space array shape:", kspace_array.shape)


# Example usage: loop over multiple patients.
pat_ids = ['0003', '0004', '0005']
pat_ids = ['0003']

for pat in pat_ids:
    process_patient(pat, roots['raw_kspace'])



Processing patient: 0003
Skipping E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00400_FID373322_t2_tse_traobl_p2_384.dat, size 915456 bytes is smaller than 1GB.
Found valid file: E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat, size: 3579.34 MB
Found 1 valid kspace files for patient 0003 in E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces
Processing E:\01_data\01_prostate_raw_kspace_umcg_p1\0003_patient_umcg_done\kspaces\meas_MID00401_FID373323_t2_tse_traobl_p2_384.dat
Software version: VD/VE (!?)

Scan  0


100%|██████████| 38.1M/38.1M [00:00<00:00, 403MB/s]


Scan  1


100%|██████████| 3.46G/3.46G [00:02<00:00, 1.61GB/s]


Number of elements in twix_obj = 2
  mdb: value to big for printing
  hdr: value to big for printing
  hdr_str: value to big for printing
  raidfile_hdr: value to big for printing
Echo train 21: [525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 525, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 527, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 531, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 533, 535, 535, 535, 535, 535, 535, 535, 535, 535, 535, 535,

# What if we just get it from the conversion loop from .mrd to h5?

In [ ]:
def build_kspace_array_from_mrd_umcg(fpath_mrd: str) -> np.ndarray:
    """
    This function builds a k-space array from a .mrd file.
    
    Parameters:
    - fpath_mrd (str): Path to the .mrd file.
    
    Returns:
    - kspace (numpy.ndarray): The k-space data.
    - echo_train_mapping (dict): A mapping of echo train numbers to their corresponding indices.
    """
    
    print(f"\tBuilding kspace array from .mrd file")

    dset   = ismrmrd.Dataset(fpath_mrd, create_if_needed=False)
    header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
    enc    = header.encoding[0]

    ncoils     = header.acquisitionSystemInformation.receiverChannels
    nslices    = enc.encodingLimits.slice.maximum + 1 if enc.encodingLimits.slice is not None else 1
    eNy        = enc.encodedSpace.matrixSize.y
    rNx        = enc.reconSpace.matrixSize.x
    eTL        = 25 if DEBUG else echo_train_length(dset, verbose=True)                           # echo train length = 25
    # eTC        = 13 if DEBUG else echo_train_count(dset, echo_train_len=eTL, verbose=True)       # echo train count = 11
    firstacq   = get_first_acquisition(dset)
    navgs      = 3 #if DEBUG else get_num_averages(firstacq=firstacq, dset=dset)
    y_min      = enc.encodingLimits.kspace_encoding_step_1.minimum  # from the .mrd header
    total_acqs = dset.number_of_acquisitions()
    logger.info(f"\t y_min: {y_min}")
    logger.info(f"\t navgs: {navgs}, nslices: {nslices}, ncoils: {ncoils}, rNx: {rNx}, eNy: {eNy}")

    kspace = np.zeros((navgs, nslices, ncoils, rNx, eNy + 1), dtype=np.complex64)
    logger.info(f"\tFilling kspace array from mrd object to shape {kspace.shape}...\n\tNum Acquisitions: {dset.number_of_acquisitions()} \n\t\tLoading... ")

    # Loop through the rest of the acquisitions and fill the data array with the kspace data
    echo_train_mapping = {}
    for acq_idx in range(firstacq, dset.number_of_acquisitions()):
        acq          = dset.read_acquisition(acq_idx)
        slice_idx    = acq.idx.slice
        enc_step     = acq.idx.kspace_encode_step_1
        avg_idx      = acq._head.idx.average
        echo_train_n = (enc_step - y_min) // eTL  # echo train number
        effective_y  = enc_step - y_min            # compute effective phase-encode index
        echo_train_mapping.setdefault(echo_train_n, []).append(enc_step)
        logger.info(f"avg idx: {avg_idx}, slice idx: {slice_idx}, enc step: {enc_step}, echo train n: {echo_train_n}, effective y: {effective_y}")
        # Each acquisition is a 2D array of shape (coil, rNx) complex
        kspace[avg_idx, slice_idx, :, :, enc_step] = acq.data

        # if slice_idx not in slices_done:
        #     slices_done.add(slice_idx)
        #     print(f"\nNew Slice={slice_idx}, enc_step={enc_step}, echo_train_n={echo_train_n} effective_y={effective_y}", end=" ")

        # if avg_idx not in averages_done:
        #     averages_done.add(avg_idx)
        #     print(f"\nNew Average={avg_idx}", end=" ")

        # if acq_idx % 1000 == 0:
        #     print(f"{acq_idx/total_acqs*100:.0f}%", end=" ", flush=True)
    return kspace, echo_train_mapping


DEBUG  = False
logger = setup_logger(Path('logs'), namepart="pat0160" , use_time=False)

kspace, echo_mapping = build_kspace_array_from_mrd_umcg(roots['0160_mrd_fpath'])
for k, v in echo_mapping.items():
    print(f"Set of {k}: {set(v)}, len: {len(set(v))}")

TypeError: unsupported operand type(s) for /: 'str' and 'str'